## Milestone 1

## Import libraries

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## GET request to list the available files

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.66 s, sys: 6.07 s, total: 10.7 s
Wall time: 1min 15s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 15.6 s, sys: 2.77 s, total: 18.4 s
Wall time: 19.9 s


In [6]:
current_path = %pwd
data_path =  current_path + "/figshare"
%ls -ltr $data_path

total 12218440
-rw-r--r--   1 tangyiyi113  staff  814041183 29 Mar 17:58 data.zip
-rw-r--r--   1 tangyiyi113  staff   95376895 29 Mar 17:58 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff   94960113 29 Mar 17:58 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff   82474546 29 Mar 17:59 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  127613760 29 Mar 17:59 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  232118894 29 Mar 17:59 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  330360682 29 Mar 17:59 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  254009247 29 Mar 17:59 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  235661418 29 Mar 17:59 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  294260911 29 Mar 17:59 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 tangyiyi113  staff  295768615 29 Mar 17:59 EC-Earth3-Veg-LR_dail

## Combining data CSVs



In [7]:
%%time
files_to_combine = glob.glob(data_path + "/*.csv")
files_to_combine.remove(data_path + "/observed_daily_rainfall_SYD.csv")
df = pd.concat(
    (pd.read_csv(file, index_col=0, parse_dates=True)
                .assign(model=re.findall(r'[^\/&\\]+(?=_daily_rainfall_NSW\.)', file)[0])
                for file in files_to_combine)
    )
df.to_csv(current_path + "/combined_data.csv")

CPU times: user 7min 25s, sys: 15.3 s, total: 7min 40s
Wall time: 7min 50s


In [8]:
print(df.shape)

(62467843, 6)


In [9]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


## Load the combined CSV to memory and perform a simple EDA

In [10]:
df.shape

(62467843, 6)

**Observations:**

This is a relatively large dataset with 62,467,843 instances and 6 features. So, running the whole data set for data wrangling and analysis may be time-consuming. 

In [11]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

**Observations:**

As the memory usage is quite large (around 3.7+ GB), running `df.info()` would not be an efficient way to extract data type information. Instead, we used `df.dtypes`, which gives desired information with minimal time. 

As we can observe from the result, there are five numerical features ("lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)") and one categorical feature ("model"). And all numerical features is the data type of `float 64`, which consumes a lot of memory; therefore, we may think about changing data types to decrease the memory usage. 

### Change data types of numerical features

In [12]:
%%time
# Change data types of numerical features
data_types = {'lat_min': 'float16', 
          'lat_max': 'float16', 
          'lon_min': 'float16', 
          'lon_max': 'float16',
          'rain (mm/day)':'float16',
          'model':'str'}
df_1 = pd.read_csv(current_path + "/combined_data.csv", index_col=0, parse_dates=True, dtype=data_types)
print(df_1["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 1min 2s, sys: 6.75 s, total: 1min 8s
Wall time: 1min 9s


In [13]:
df_1.dtypes

lat_min          float16
lat_max          float16
lon_min          float16
lon_max          float16
rain (mm/day)    float16
model             object
dtype: object

In [14]:
len(df_1["model"].unique())

27

In [15]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float16: {df_1.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3498.20 MB
Memory usage with float16: 1624.16 MB


**Observations:**

By changing the data types of numerical features from `float 64` to `float 16`, we successfully reduce the memory usage by almost a half. However, we are not able to know the most suitable data type for each column before changing it.

There are 27 unique models in the data frame, and each of them has at least 550,000 instances. Additionally, the model has the highest number of examples is MPI-ESM1-2-HR, and the model with the lowest number of examples is CanESM5

### Load just columns what we want

In [16]:
%%time
# Choose columns, which are needed
use_cols = ['rain (mm/day)', 'model']
df_2 = pd.read_csv(current_path + "/combined_data.csv", usecols=use_cols)
print(df_2["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 30.9 s, sys: 2.14 s, total: 33 s
Wall time: 33.1 s


In [17]:
print(f"Original memory usage: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage after loading selected columns: {df_2.memory_usage().sum() / 1e6:.2f} MB")

Original memory usage: 3498.20 MB
Memory usage after loading selected columns: 999.49 MB


**Observations:**

As our main interest is the rainfall, so in order to speed up the process, we only load the needed columns. By reducing the number of columns, we are able to reduce the time to 34.5 sec, which is a big improvement on processing time. Also, the memory usage is way lower than the original data frame. However, we do not know which column would be unnecessary for future analysis. For example, if we would like to perform spatial visualization for rainfall, then latitude and longitude may be useful.

### Process data in chunks

In [18]:
%%time
#process data in chunks
df_3 = pd.Series(dtype=float)
for chunk in pd.read_csv(current_path + "/combined_data.csv", chunksize=10_000_000):
    df_3 = df_3.add(chunk["model"].value_counts(), fill_value=0)
print(df_3.astype(float))

ACCESS-CM2          1932840.0
ACCESS-ESM1-5       1610700.0
AWI-ESM-1-1-LR       966420.0
BCC-CSM2-MR         3035340.0
BCC-ESM1             551880.0
CMCC-CM2-HR4        3541230.0
CMCC-CM2-SR5        3541230.0
CMCC-ESM2           3541230.0
CanESM5              551880.0
EC-Earth3-Veg-LR    3037320.0
FGOALS-f3-L         3219300.0
FGOALS-g3           1287720.0
GFDL-CM4            3219300.0
GFDL-ESM4           3219300.0
INM-CM4-8           1609650.0
INM-CM5-0           1609650.0
KIOST-ESM           1287720.0
MIROC6              2070900.0
MPI-ESM-1-2-HAM      966420.0
MPI-ESM1-2-HR       5154240.0
MPI-ESM1-2-LR        966420.0
MRI-ESM2-0          3037320.0
NESM3                966420.0
NorESM2-LM           919800.0
NorESM2-MM          3541230.0
SAM0-UNICON         3541153.0
TaiESM1             3541230.0
dtype: float64
CPU times: user 55.6 s, sys: 7.45 s, total: 1min 3s
Wall time: 1min 3s


**Observations:**

By loading data in chunks with chunk size of 10 millions, we can reduce the loading time to 1min and 3s!

### Perform a simple EDA in R

In [19]:
#See your summarized stats
print('Total views =', df_formatted['views'].sum(),'and total downloads=',df_formatted['downloads'].sum())


NameError: name 'df_formatted' is not defined

## Compare performance

#### Performance for Combining Data CSV

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Joyce Wang  |     Windows      |16GB | 11th Gen Intel(R) Core(TM) i7-1165G7 2.80GHz | yes| 9min 29s   |
| Macy Chan   |              |     |        |        |            |
| Steven Leung|          Mac           |  16GB   |     Apple M1      |    yes    |     7min 25s       |
| Zack Tang   |   Mac         |   16GB  |       2.2 GHz 6-Core Intel Core i7    |    yes    |    7min 25s |      


#### Performance for EDA in python (Changing data types)

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Joyce Wang  |                  |     |           |        |            |
| Macy Chan   |                  |     |           |        |            |
| Steven Leung|       Mac           |  16GB   |     Apple M1      |    yes    |     1min 2s       |
| Zack Tang   |     Mac         |   16GB  |       2.2 GHz 6-Core Intel Core i7    |    yes    | 1min 2s |

#### Performance for EDA in python (Loading what is needed)

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Joyce Wang  |                  |     |           |        |            |
| Macy Chan   |                  |     |           |        |            |
| Steven Leung|      Mac           |  16GB   |     Apple M1      |    yes    |     30.3s       |
| Zack Tang   |     Mac         |   16GB  |       2.2 GHz 6-Core Intel Core i7    |    yes    | 30.9s |

#### Performance for EDA in python (Loading data in chunks)

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Joyce Wang  |                  |     |           |        |            |
| Macy Chan   |                  |     |           |        |            |
| Steven Leung|      Mac           |  16GB   |     Apple M1      |    yes    |     52.3s       |
| Zack Tang   |     Mac         |   16GB  |       2.2 GHz 6-Core Intel Core i7    |    yes    | 1min 3s |

